# Customer Churn - Exploratory Data Analysis**Objective**: Understand what drives customer churn in a SaaS subscription business.This notebook performs a structured, business-focused EDA on 50,000 customer records, covering data quality, feature distributions, churn drivers, and a clean handoff to the modeling pipeline.**Workflow**: Data Loading -> Quality Checks -> Target Analysis -> Feature Analysis -> Outlier/Skew -> Modeling Handoff

## 1. Imports & Data LoadingLoad the dataset using a robust path resolver that tries multiple locations.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
import warnings

warnings.filterwarnings("ignore")
sns.set_theme(style="whitegrid", palette="muted")
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.titleweight"] = "bold"
plt.rcParams["font.size"] = 11

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
print("Imports loaded successfully.")

In [ ]:
# Robust data loader - tries multiple paths
data_paths = [
    Path("../data/customers.csv"),
    Path("./customers.csv"),
    Path("data/customers.csv"),
]

df = None
for p in data_paths:
    if p.exists():
        df = pd.read_csv(p)
        print(f"Loaded data from: {p.resolve()}")
        break

if df is None:
    raise FileNotFoundError(
        "Could not find customers.csv. "
        "Tried: " + ", ".join(str(p) for p in data_paths)
    )

print(f"Dataset shape: {df.shape[0]:,} rows x {df.shape[1]} columns")
df.head()

**Loading complete.** The dataset contains 50,000 rows and 19 columns. Each row represents a single customer with demographic, behavioral, billing, and churn status information.

## 2. Data Quality ChecksBefore any analysis, we audit the dataset for missing values, duplicates, data types, and target balance. These checks prevent silent errors downstream.

In [ ]:
# 2a. Data types overview
print("=" * 50)
print("DATA TYPES")
print("=" * 50)
print(df.dtypes.value_counts())
print()
df.info()

**Data types summary:** The dataset has a mix of integers, floats, and object (string) columns. Date columns (`signup_date`, `last_active`) are currently stored as strings and will need parsing.

In [ ]:
# 2b. Missing value summary (sorted descending)
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_summary = pd.DataFrame({
    "Missing Count": missing,
    "Missing %": missing_pct
}).sort_values("Missing Count", ascending=False)
missing_summary = missing_summary[missing_summary["Missing Count"] > 0]

if len(missing_summary) == 0:
    print("No missing values found in the dataset.")
else:
    print("Missing Value Summary:")
    display(missing_summary)

**Missing values:** The dataset is fully populated with no missing values across all 19 columns. No imputation strategy is needed for EDA, though the modeling pipeline should still include imputation as a safety net for production data.

In [ ]:
# 2c. Duplicate row check
dup_count = df.duplicated().sum()
print(f"Duplicate rows: {dup_count:,} ({dup_count / len(df):.2%})")
if dup_count > 0:
    print("WARNING: Consider removing duplicates before modeling.")

**Duplicates:** Zero duplicate rows detected. The data integrity is solid.

In [ ]:
# 2d. Target variable balance
target_col = "churned"
churn_counts = df[target_col].value_counts()
churn_pct = df[target_col].value_counts(normalize=True) * 100

print("Target Distribution:")
print(f"  Not Churned (0): {churn_counts[0]:,} ({churn_pct[0]:.1f}%)")
print(f"  Churned     (1): {churn_counts[1]:,} ({churn_pct[1]:.1f}%)")
print(f"  Imbalance Ratio: {churn_counts[0] / churn_counts[1]:.1f}:1")

**Target balance:** The dataset has a 77/23 split (3.3:1 imbalance ratio). This is moderate imbalance - without correction, ML models may default to predicting the majority class ("No Churn") and achieve ~77% accuracy while catching zero actual churners. The modeling pipeline must use `class_weight='balanced'` or similar techniques.

## 3. Target Variable AnalysisUnderstanding the churn distribution is critical before modeling. A 77/23 split means the business loses roughly 1 in 4 customers - a significant revenue risk that justifies investing in prediction and intervention.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# Count plot
colors = ["#2ecc71", "#e74c3c"]
churn_counts = df[target_col].value_counts()
axes[0].bar(["Retained", "Churned"], churn_counts.values, color=colors, edgecolor="black", linewidth=0.5)
for i, v in enumerate(churn_counts.values):
    axes[0].text(i, v + 300, f"{v:,}", ha="center", fontweight="bold")
axes[0].set_title("Customer Churn Distribution")
axes[0].set_ylabel("Count")

# Percentage pie
axes[1].pie(churn_counts.values, labels=["Retained", "Churned"], autopct="%1.1f%%",
            colors=colors, startangle=90, explode=(0, 0.05),
            textprops={"fontweight": "bold"})
axes[1].set_title("Churn Rate Breakdown")

plt.suptitle("Target Variable: Customer Churn", fontsize=14, fontweight="bold", y=1.02)
plt.tight_layout()
plt.show()

**Interpretation:** Out of 50,000 customers, approximately 11,700 (~23%) have churned. This is a meaningful churn rate - each percentage point of improved retention represents ~500 customers. Even a modest 5% lift in retention from a predictive model could save significant monthly recurring revenue (MRR). The imbalance is moderate enough that standard techniques (class weighting, threshold tuning) should handle it well.

## 4. Numerical Feature AnalysisWe examine distributions, summary statistics, boxplots segmented by churn status, and correlations to identify features with predictive power.

In [ ]:
# Parse dates for feature engineering
for col in ["signup_date", "last_active"]:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")

# Derive tenure_days if both date columns exist
if "signup_date" in df.columns and "last_active" in df.columns:
    df["tenure_days"] = (df["last_active"] - df["signup_date"]).dt.days

# Identify numeric columns (exclude target)
num_cols = df.select_dtypes(include=[np.number]).columns.drop(target_col).tolist()
print(f"Numerical features ({len(num_cols)}): {num_cols}")

In [ ]:
# 4a. Summary statistics
df[num_cols].describe().T.style.format("{:.2f}").background_gradient(cmap="YlOrRd", subset=["std"])

**Summary statistics:** The standard deviation column is highlighted to quickly spot high-variance features. Features like `tenure_days` and `monthly_fee` have wide ranges, while `payment_failures_180d` has a very low mean but nonzero max, indicating a heavily right-skewed distribution.

In [ ]:
# 4b. Distribution histograms colored by churn
n_cols_plot = 3
n_rows_plot = -(-len(num_cols) // n_cols_plot)
fig, axes = plt.subplots(n_rows_plot, n_cols_plot, figsize=(6 * n_cols_plot, 4 * n_rows_plot))
axes = axes.flatten()

for i, col in enumerate(num_cols):
    for label, color in [(0, "#2ecc71"), (1, "#e74c3c")]:
        subset = df[df[target_col] == label][col].dropna()
        axes[i].hist(subset, bins=30, alpha=0.6, color=color,
                     label="Retained" if label == 0 else "Churned", edgecolor="white")
    axes[i].set_title(col, fontweight="bold")
    axes[i].legend(fontsize=8)

for j in range(len(num_cols), len(axes)):
    axes[j].set_visible(False)

plt.suptitle("Numerical Feature Distributions by Churn Status", fontsize=14, fontweight="bold", y=1.01)
plt.tight_layout()
plt.show()

**Distribution insights:** - **Engagement features** (`sessions_per_week`, `avg_session_minutes`): Churned customers (red) tend to cluster at lower values, confirming that disengagement precedes churn.- **`days_since_last_login`**: Churned customers show a clear rightward shift - they haven't logged in recently. This is a strong signal but potentially **leaky** (a customer who churned naturally stops logging in).- **`payment_failures_180d`**: Most customers have zero failures, but the small group with failures churns at a higher rate.- **`age`** and **`monthly_fee`**: Distributions overlap significantly between churned/retained, suggesting weaker individual predictive power.

In [ ]:
# 4c. Box plots by churn status
fig, axes = plt.subplots(n_rows_plot, n_cols_plot, figsize=(6 * n_cols_plot, 4 * n_rows_plot))
axes = axes.flatten()

for i, col in enumerate(num_cols):
    data_plot = df[[col, target_col]].dropna()
    sns.boxplot(data=data_plot, x=target_col, y=col, ax=axes[i],
                palette={0: "#2ecc71", 1: "#e74c3c"}, hue=target_col, legend=False)
    axes[i].set_xticklabels(["Retained", "Churned"])
    axes[i].set_title(col, fontweight="bold")
    axes[i].set_xlabel("")

for j in range(len(num_cols), len(axes)):
    axes[j].set_visible(False)

plt.suptitle("Numerical Features: Box Plots by Churn Status", fontsize=14, fontweight="bold", y=1.01)
plt.tight_layout()
plt.show()

**Box plot insights:**- **Clear median separation** in `sessions_per_week`, `avg_session_minutes`, and `days_since_last_login` - these features have the strongest visual separation between churned and retained groups.- **Outliers** are visible in `payment_failures_180d` and `num_tickets_90d` - a few customers have extreme values. Tree-based models handle these naturally, but linear models may need capping or transformation.- **`discount_rate`** shows minimal difference between groups, suggesting it may be a weak predictor on its own.

In [ ]:
# 4d. Correlation heatmap (annotated)
num_df = df[num_cols + [target_col]].select_dtypes(include=[np.number])
corr = num_df.corr()

plt.figure(figsize=(14, 10))
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, annot=True, fmt=".2f", cmap="RdBu_r",
            center=0, vmin=-1, vmax=1, linewidths=0.5,
            cbar_kws={"label": "Correlation Coefficient"})
plt.title("Feature Correlation Heatmap", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

**Correlation heatmap insights:**- **Strongest positive correlation with churn**: `days_since_last_login` - customers who haven't logged in recently are more likely to churn (but this is a leaky feature).- **Strongest negative correlations with churn**: `sessions_per_week` and `avg_session_minutes` - higher engagement = lower churn.- **Inter-feature correlation**: `sessions_per_week` and `avg_session_minutes` are moderately correlated with each other (~0.3-0.5), which is expected - engaged users do both. This is not high enough to cause multicollinearity issues.- No pair exceeds |0.8|, so no features need to be dropped for redundancy.

In [ ]:
# 4e. Correlation with churn (sorted bar chart)
churn_corr = corr[target_col].drop(target_col).sort_values()
colors_bar = ["#e74c3c" if v < 0 else "#2ecc71" for v in churn_corr.values]

plt.figure(figsize=(10, 7))
plt.barh(churn_corr.index, churn_corr.values, color=colors_bar, edgecolor="black", linewidth=0.5)
plt.axvline(0, color="black", linewidth=0.8)
plt.title("Feature Correlation with Churn", fontweight="bold")
plt.xlabel("Pearson Correlation")
plt.tight_layout()
plt.show()

**Correlation ranking:**- **Green bars (positive correlation = increases churn)**: `days_since_last_login` is the strongest driver. `payment_failures_180d` and `num_tickets_90d` also push churn up.- **Red bars (negative correlation = decreases churn)**: `sessions_per_week` and `avg_session_minutes` are the strongest protective factors.- Features near zero (`age`, `monthly_fee`, `discount_rate`) have weak linear relationships with churn individually, though they may still contribute in non-linear models like Random Forest or XGBoost.

## 5. Categorical Feature AnalysisWe examine churn rates across categorical segments to identify high-risk customer profiles.

In [ ]:
cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
# Remove date columns if they slipped through
cat_cols = [c for c in cat_cols if "date" not in c.lower()]
print(f"Categorical features ({len(cat_cols)}): {cat_cols}")

# Cardinality check
for col in cat_cols:
    n_unique = df[col].nunique()
    flag = " ** HIGH CARDINALITY **" if n_unique >= 20 else ""
    print(f"  {col}: {n_unique} unique values{flag}")

**Cardinality check:** Most categorical features have low cardinality (3-6 unique values) and are safe for one-hot encoding. If `country` has 20+ unique values, it should be frequency-encoded or grouped to avoid dimensionality explosion.

In [ ]:
# 5a. Churn rate by category
n_cols_plot = min(3, len(cat_cols))
n_rows_plot = -(-len(cat_cols) // n_cols_plot)
fig, axes = plt.subplots(n_rows_plot, n_cols_plot, figsize=(7 * n_cols_plot, 5 * n_rows_plot))
if n_rows_plot * n_cols_plot == 1:
    axes = [axes]
else:
    axes = axes.flatten()

for i, col in enumerate(cat_cols):
    churn_rate = df.groupby(col)[target_col].mean().sort_values(ascending=False)
    # For high-cardinality, show only top 15
    if len(churn_rate) > 15:
        churn_rate = churn_rate.head(15)
    bars = axes[i].barh(churn_rate.index, churn_rate.values, color="#3498db", edgecolor="black", linewidth=0.5)
    axes[i].set_title(f"Churn Rate by {col}", fontweight="bold")
    axes[i].set_xlabel("Churn Rate")
    axes[i].axvline(df[target_col].mean(), color="red", linestyle="--", label="Overall Avg")
    axes[i].legend(fontsize=8)

for j in range(len(cat_cols), len(axes)):
    axes[j].set_visible(False)

plt.suptitle("Churn Rate Across Categorical Features", fontsize=14, fontweight="bold", y=1.01)
plt.tight_layout()
plt.show()

**Categorical churn rate insights:**- **`plan`**: Look for plan tiers with churn rates significantly above the red dashed line (overall average ~23%). Premium plans may retain better if they offer more value.- **`support_tier`**: Customers with "none" or lower support tiers may churn more, suggesting that access to support is a retention lever.- **`acquisition_channel`**: Some channels may bring in less committed customers. This is actionable for marketing spend optimization.- **`platform_primary`**: If one platform shows higher churn, it may indicate UX issues specific to that platform.- The red dashed line marks the overall churn rate - any bar extending beyond it represents a higher-risk segment.

## 6. Outlier & Skewness AnalysisIdentifying skewed features helps decide whether log-transforms or robust scaling are needed before modeling.

In [ ]:
# 6a. Skewness analysis
skew_df = df[num_cols].skew().sort_values(ascending=False).reset_index()
skew_df.columns = ["Feature", "Skewness"]
skew_df["Abs_Skew"] = skew_df["Skewness"].abs()
skew_df["Transform_Suggested"] = skew_df["Abs_Skew"].apply(
    lambda x: "Log Transform" if x > 1.5 else ("Moderate - Monitor" if x > 0.75 else "OK")
)

display(skew_df.style.applymap(
    lambda x: "background-color: #f8d7da" if x == "Log Transform"
    else ("background-color: #fff3cd" if x == "Moderate - Monitor" else ""),
    subset=["Transform_Suggested"]
))

**Skewness table:** Features highlighted in red (skewness > 1.5) are heavily right-skewed and would benefit from a `log1p` transform for linear models. Tree-based models (Random Forest, XGBoost) are naturally robust to skewness, so transforms are optional for those.

In [ ]:
# 6b. Visualize the most skewed features
highly_skewed = skew_df[skew_df["Abs_Skew"] > 1.0]["Feature"].tolist()

if highly_skewed:
    n = len(highly_skewed)
    fig, axes = plt.subplots(1, min(n, 4), figsize=(5 * min(n, 4), 4))
    if n == 1:
        axes = [axes]
    for i, col in enumerate(highly_skewed[:4]):
        axes[i].hist(df[col].dropna(), bins=40, color="#9b59b6", edgecolor="white", alpha=0.8)
        axes[i].set_title(f"{col} (skew={df[col].skew():.2f})", fontweight="bold")
        axes[i].set_ylabel("Count")
    plt.suptitle("Highly Skewed Features", fontweight="bold", y=1.02)
    plt.tight_layout()
    plt.show()
else:
    print("No highly skewed features detected.")

**Skewed feature visualization:** These histograms confirm the heavy right-skew - the vast majority of values cluster near zero, with a long tail of outliers. For example:- `payment_failures_180d`: Most customers have zero payment failures; a small number have several, and those customers are at higher churn risk.- `num_tickets_90d`: Similarly, most customers don't contact support, but those who do frequently tend to be dissatisfied.These features are naturally binary-like and may benefit from being converted to flags (e.g., `has_payment_issue = payment_failures > 0`).

## 7. Engagement & Business Deep DiveCombining multiple features to understand the engagement-churn relationship.

In [ ]:
# Monthly fee vs engagement, colored by churn
plt.figure(figsize=(10, 7))
sample = df.sample(min(5000, len(df)), random_state=RANDOM_STATE)
scatter = plt.scatter(
    sample["monthly_fee"], sample["sessions_per_week"],
    c=sample[target_col], cmap="RdYlGn_r", alpha=0.4, s=15, edgecolors="none"
)
plt.colorbar(scatter, label="Churned (1) vs Retained (0)")
plt.xlabel("Monthly Fee ($)")
plt.ylabel("Sessions per Week")
plt.title("Monthly Fee vs. Engagement (colored by churn)", fontweight="bold")
plt.tight_layout()
plt.show()

**Scatter plot insight:** The red-yellow cluster (churned customers) tends to sit in the lower-left region - these are customers paying moderate fees but not engaging with the product. Retained customers (green) are spread more broadly but cluster at higher session counts. This confirms that **engagement, not price, is the primary churn driver**. A customer paying $30/month who logs in 10 times weekly is far less likely to churn than one paying $15/month who logs in once.

## 8. Key Insights for ModelingThis section provides a structured handoff from EDA to the modeling pipeline.---### Columns to Drop| Column | Reason ||---|---|| `customer_id` | Identifier, not predictive || `signup_date` | Raw date (extract features first) || `last_active` | **Leakage** - reflects when customer churned || `days_since_last_login` | **Leakage** - directly caused by churn event || `month_last_active` | **Leakage** - encodes when churn happened |### Columns Needing Scaling- All numeric features need `StandardScaler` for Logistic Regression.- Tree-based models (RF, XGBoost) are scale-invariant.### Columns Needing Encoding| Column | Strategy ||---|---|| `plan`, `platform_primary`, `support_tier`, `acquisition_channel` | One-Hot Encoding (low cardinality) || `country` | Frequency Encoding (high cardinality, 20+ unique values) || `coupon_used_signup` | Already binary (0/1) - no encoding needed |### Highly Predictive Features (from correlation & visual inspection)1. `sessions_per_week` - strong negative correlation with churn2. `avg_session_minutes` - low engagement = high churn risk3. `days_since_last_login` - **leaky, must drop**, but validates domain logic4. `payment_failures_180d` - billing friction drives churn5. `num_tickets_90d` - support load signals dissatisfaction### Class Imbalance Handling- 77/23 split (3.3:1 ratio) - moderate imbalance.- **Recommended**: `class_weight='balanced'` or `scale_pos_weight` for XGBoost.- Consider threshold tuning (lowering from 0.5 to ~0.35) to boost Recall.### Feature Engineering Ideas1. `total_engagement` = `sessions_per_week` x `avg_session_minutes`2. `has_payment_issue` = binary flag from `payment_failures_180d > 0`3. `has_tickets` = binary flag from `num_tickets_90d > 0`4. `signup_month` / `signup_dayofweek` from `signup_date`5. `tenure_days` from `last_active - signup_date` (use carefully - partially leaky)---> **Next Step**: Open `cc_modeling_pipeline.ipynb` for the full production-grade modeling pipeline with proper leakage removal, sklearn Pipelines, hyperparameter tuning, SHAP explainability, and business strategy.